In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import time

sample = np.load('dataset.npy')
print(sample.shape)

(105829, 1025)


In [2]:
idx = np.random.choice(sample.shape[0], sample.shape[0], replace=False).tolist()
dataset = sample[idx]
batch_size = 64
Size = int(dataset.shape[0] * 0.9)
Train = dataset[:Size, :]
Train = np.vstack((Train, np.zeros((batch_size - Train.shape[0] % batch_size, Train.shape[1]))))
Train = np.reshape(Train,(-1, batch_size, Train.shape[1]))
Test = dataset[-(dataset.shape[0] - Size):, :]
Test = np.vstack((Test, np.zeros((batch_size - Test.shape[0] % batch_size, Test.shape[1]))))
Test = np.reshape(Test,(-1, batch_size, Test.shape[1]))
print(Train.shape)
print(Test.shape)

num_epoch = 20

(1489, 64, 1025)
(166, 64, 1025)


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

unit_size = [8, 16, 32, 64, 128, 192]
fc_size = [120,240,480,960,1920,2880]
for item,fc_size in zip(unit_size,fc_size):
    class Net(nn.Module):
        def __init__(self, in_channels, out_channels, unit_size = 32):
            super().__init__()
            self.fc1 = nn.Conv1d(in_channels, unit_size, kernel_size=32, stride=4)
            self.bn1 = nn.BatchNorm1d(unit_size)
            self.pool1 = nn.MaxPool1d(4)
            self.fc2 = nn.Conv1d(unit_size, unit_size, kernel_size=3, padding = 1)
            self.bn2 = nn.BatchNorm1d(unit_size)
            self.fc3 = nn.Conv1d(unit_size, unit_size, kernel_size=3, padding = 1)
            self.bn3 = nn.BatchNorm1d(unit_size)
            self.fc4 = nn.Conv1d(unit_size, unit_size, kernel_size=3, padding = 1)
            self.bn4 = nn.BatchNorm1d(unit_size)
            self.pool2 = nn.MaxPool1d(4)
            self.fc5 = nn.Conv1d(unit_size, unit_size, kernel_size=3, padding = 1)
            self.bn5 = nn.BatchNorm1d(unit_size)
            self.pool3 = nn.MaxPool1d(2)
            self.fc7 = nn.Linear(fc_size, out_channels)
        
        def forward(self, x):
            x = self.fc1(x)
            x = F.relu(self.bn1(x))
            x = self.fc2(x)
            y = self.pool1(F.relu(self.bn2(x)))
            x = self.fc3(y)
            x = F.relu(self.bn3(x))
            x = self.fc4(x + y)
            y = self.pool2(F.relu(self.bn4(x)))
            x = self.fc5(y)
            x = F.relu(self.bn5(x))
            x = torch.flatten(x, start_dim = 1)
            return F.log_softmax(self.fc7(x), dim = 1)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(1, 36, item).to(device)

    summary(model,(1,1024))

    optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = .0001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)
    criterion = nn.NLLLoss()

    max_loss = 20
    for epoch in range(num_epoch):
        model.train()
        for item in Train:
            sample = torch.unsqueeze(torch.Tensor(item[:,:-1].astype('float32')).to(device), dim = 1)
            label = torch.LongTensor(item[:,-1].astype('int')).to(device)
            output = model(sample)
            loss = criterion(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
        if(loss.item() < max_loss):
            torch.save(model.state_dict(), 'best.pt')
            max_loss = loss.item()

    model.load_state_dict(torch.load('best.pt'))
    correct = 0
    count = 0
    start = time.time()
    for item in Test:
        sample = torch.unsqueeze(torch.Tensor(item[:,:-1].astype('float32')).to(device), dim = 1)
        label = item[:,-1].astype('int')
        output = model(sample)
        output = torch.argmax(output, dim = 1)
        tmp = [1 if i.item() == j else 0 for i,j in zip(output, label)]
        correct += np.sum(np.asarray(tmp))
        count += label.shape[0]
    
    print(f'Accuracy: {correct * 100 / count : .6f} %, Average Inference Time: {(start - time.time())/ len(Test):.6f}')

        

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 8, 249]             264
       BatchNorm1d-2               [-1, 8, 249]              16
            Conv1d-3               [-1, 8, 249]             200
       BatchNorm1d-4               [-1, 8, 249]              16
         MaxPool1d-5                [-1, 8, 62]               0
            Conv1d-6                [-1, 8, 62]             200
       BatchNorm1d-7                [-1, 8, 62]              16
            Conv1d-8                [-1, 8, 62]             200
       BatchNorm1d-9                [-1, 8, 62]              16
        MaxPool1d-10                [-1, 8, 15]               0
           Conv1d-11                [-1, 8, 15]             200
      BatchNorm1d-12                [-1, 8, 15]              16
           Linear-13                   [-1, 36]           4,356
Total params: 5,500
Trainable params: 5

Accuracy:  77.804970 %, Average Inference Time: -0.002537


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
unit_size = [2048,1024,512,256]

for item in unit_size:
    class Net(nn.Module):
        def __init__(self, in_channels, out_channels, unit_size = 512):
            super().__init__()
            self.fc1 = nn.Linear(in_channels, unit_size)
            self.bn1 = nn.BatchNorm1d(1)
            self.pool1 = nn.MaxPool1d(4)
            self.fc2 = nn.Linear(unit_size,unit_size)
            self.bn2 = nn.BatchNorm1d(1)
            self.fc3 = nn.Linear(unit_size//4, unit_size//4)
            self.bn3 = nn.BatchNorm1d(1)
            self.fc4 = nn.Linear(unit_size//4, unit_size//4)
            self.bn4 = nn.BatchNorm1d(1)
            self.pool2 = nn.MaxPool1d(2)
            self.fc5 = nn.Linear(unit_size//8, unit_size// 8)
            self.bn5 = nn.BatchNorm1d(1)
            self.pool3 = nn.MaxPool1d(2)
            self.fc7 = nn.Linear(unit_size//8, out_channels)
        
        def forward(self, x):
            x = self.fc1(x)
            x = F.relu(self.bn1(x))
            x = self.fc2(x)
            y = self.pool1(F.relu(self.bn2(x)))
            x = self.fc3(y)
            x = F.relu(self.bn3(x))
            x = self.fc4(x + y)
            y = self.pool2(F.relu(self.bn4(x)))
            x = self.fc5(y)
            x = F.relu(self.bn5(x))
            x = torch.flatten(x, start_dim = 1)
            return F.log_softmax(self.fc7(x), dim = 1)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(1024, 36, item).to(device)

    summary(model,(1,1024))

    optimizer = optim.Adam(model.parameters(), lr = 0.005, weight_decay = .0001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)
    criterion = nn.NLLLoss()
    max_loss = 20

    for epoch in range(num_epoch):
        model.train()
        for item in Train:
            sample = torch.unsqueeze(torch.Tensor(item[:,:-1].astype('float32')).to(device), dim = 1)
            label = torch.LongTensor(item[:,-1].astype('int')).to(device)
            output = model(sample)
            loss = criterion(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
        if(loss.item() < max_loss):
            torch.save(model.state_dict(), 'best.pt')
            max_loss = loss.item()

    model.load_state_dict(torch.load('best.pt'))
    
    correct = 0
    count = 0
    start = time.time()
    for item in Test:
        sample = torch.unsqueeze(torch.Tensor(item[:,:-1].astype('float32')).to(device), dim = 1)
        label = item[:,-1].astype('int')
        output = model(sample)
        output = torch.argmax(output, dim = 1)
        tmp = [1 if i.item() == j else 0 for i,j in zip(output, label)]
        correct += np.sum(np.asarray(tmp))
        count += label.shape[0]
    
    print(f'Accuracy: {correct * 100 / count : .6f} %, Average Inference Time: {(start - time.time())/ len(Test):.6f}')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 1, 2048]       2,099,200
       BatchNorm1d-2              [-1, 1, 2048]               2
            Linear-3              [-1, 1, 2048]       4,196,352
       BatchNorm1d-4              [-1, 1, 2048]               2
         MaxPool1d-5               [-1, 1, 512]               0
            Linear-6               [-1, 1, 512]         262,656
       BatchNorm1d-7               [-1, 1, 512]               2
            Linear-8               [-1, 1, 512]         262,656
       BatchNorm1d-9               [-1, 1, 512]               2
        MaxPool1d-10               [-1, 1, 256]               0
           Linear-11               [-1, 1, 256]          65,792
      BatchNorm1d-12               [-1, 1, 256]               2
           Linear-13                   [-1, 36]           9,252
Total params: 6,895,918
Trainable param

In [5]:
#Conv1d classifier
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
channels = [8,16,32,64,128]
fc_size = [288,576,1152,2304,4608]
for c,f in zip(channels,fc_size):
    class Net(nn.Module):
        def __init__(self, out_channels, channels = 8):
            super().__init__()
            self.fc1 = nn.Conv2d(1, channels, kernel_size=5, stride=2, padding = 1)
            self.bn1 = nn.BatchNorm2d(channels)
            self.pool1 = nn.MaxPool2d(2)
            self.fc2 = nn.Conv2d(channels, channels * 2, kernel_size=3, padding = 1)
            self.bn2 = nn.BatchNorm2d(channels*2)
            self.fc3 = nn.Conv2d(channels * 2, channels * 2, kernel_size=3, padding = 1)
            self.bn3 = nn.BatchNorm2d(channels*2)
            self.fc4 = nn.Conv2d(channels * 2, channels * 2, kernel_size=3, padding = 1)
            self.bn4 = nn.BatchNorm2d(channels * 2)
            self.pool2 = nn.MaxPool2d(2)
            self.fc5 = nn.Conv2d(channels * 2, channels * 4, kernel_size=3, padding = 1)
            self.bn5 = nn.BatchNorm2d(channels *4)
            self.pool3 = nn.MaxPool2d(2)
            self.fc7 = nn.Linear(f, out_channels)
        
        def forward(self, x):
            x = self.fc1(x)
            x = F.relu(self.bn1(x))
            x = self.fc2(x)
            y = self.pool1(F.relu(self.bn2(x)))
            x = self.fc3(y)
            x = F.relu(self.bn3(x))
            x = self.fc4(x + y)
            y = self.pool2(F.relu(self.bn4(x)))
            x = self.fc5(y)
            x = F.relu(self.bn5(x))
            x = torch.flatten(x, start_dim = 1)
            return F.log_softmax(self.fc7(x), dim = 1)
        
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(36,c).to(device)

    summary(model,(1,32,32))

    optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = .0001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)
    criterion = nn.NLLLoss()
    max_loss = 20

    for epoch in range(num_epoch):
        model.train()
        for item in Train:
            sample = torch.unsqueeze(torch.Tensor(np.reshape(item[:,:-1],(64,32,32)).astype('float32')).to(device), dim = 1)
            label = torch.LongTensor(item[:,-1].astype('int')).to(device)
            output = model(sample)
            loss = criterion(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
        if(loss.item() < max_loss):
            torch.save(model.state_dict(), 'best.pt')
            max_loss = loss.item()

    model.load_state_dict(torch.load('best.pt'))
    
    correct = 0
    count = 0
    start = time.time()
    for item in Test:
        sample = torch.unsqueeze(torch.Tensor(np.reshape(item[:,:-1],(64,32,32)).astype('float32')).to(device), dim = 1)
        label = item[:,-1].astype('int')
        output = model(sample)
        output = torch.argmax(output, dim = 1)
        tmp = [1 if i.item() == j else 0 for i,j in zip(output, label)]
        correct += np.sum(np.asarray(tmp))
        count += label.shape[0]
    
    print(f'Accuracy: {correct * 100 / count : .6f} %, Average Inference Time: {(start - time.time())/ len(Test):.6f}')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 15, 15]             208
       BatchNorm2d-2            [-1, 8, 15, 15]              16
            Conv2d-3           [-1, 16, 15, 15]           1,168
       BatchNorm2d-4           [-1, 16, 15, 15]              32
         MaxPool2d-5             [-1, 16, 7, 7]               0
            Conv2d-6             [-1, 16, 7, 7]           2,320
       BatchNorm2d-7             [-1, 16, 7, 7]              32
            Conv2d-8             [-1, 16, 7, 7]           2,320
       BatchNorm2d-9             [-1, 16, 7, 7]              32
        MaxPool2d-10             [-1, 16, 3, 3]               0
           Conv2d-11             [-1, 32, 3, 3]           4,640
      BatchNorm2d-12             [-1, 32, 3, 3]              64
           Linear-13                   [-1, 36]          10,404
Total params: 21,236
Trainable params: 